## Homework: Agents

In this homework, we will learn more about function calling,
and we will also explore MCP - model-context protocol. 


## Preparation

First, we'll define a function that we will use when building
our agent. 

It will generate fake weather data:

In [1]:
import random

known_weather_data = {
    'berlin': 20.0
}

def get_weather(city: str) -> float:
    city = city.strip().lower()

    if city in known_weather_data:
        return known_weather_data[city]

    return round(random.uniform(-5, 35), 1)

## Q1. Define function description

We want to use it as a tool for our agent, so we need to 
describe it 

How should the description for this function look like? Fill in missing parts

```python
get_weather_tool = {
    "type": "function",
    "name": "<TODO1>",
    "description": "<TODO2>",
    "parameters": {
        "type": "object",
        "properties": {
            "<TODO3>": {
                "type": "string",
                "description": "<TODO4>"
            }
        },
        "required": [TODO5],
        "additionalProperties": False
    }
}
```

What did you put in `TODO3`?

> `city`

In [2]:
get_weather_tool = {
    "type": "function",                       # ← tells the model this is a callable tool
    "name": "get_weather",                    # TODO1
    "description": "Return the current temperature in °C for a given city.",  # TODO2
    "parameters": {
        "type": "object",
        "properties": {
            "city": {                         # TODO3
                "type": "string",
                "description": "Name of the city to retrieve the weather for."  # TODO4
            }
        },
        "required": ["city"],                 # TODO5
        "additionalProperties": False
    }
}


***Why these choices***

| Field                         | Value chosen                         | Reason                                                                                                                |
| ----------------------------- | ------------------------------------ | --------------------------------------------------------------------------------------------------------------------- |
| `name`                        | `"get_weather"`                      | Must exactly match your Python function’s name so the orchestrator can link the model’s call to the code.             |
| `description`                 | Clear, one-sentence goal (“Return…”) | The model uses this natural-language text to decide *when* and *how* to call the tool. Clarity reduces hallucination. |
| `properties.city`             | `"city"`                             | The sole input your function needs. Keeping the schema minimal prevents the model from inventing extra keys.          |
| `properties.city.description` | Explains the value                   | Helps the model format the argument correctly (plain city name, no extra stuff).                                      |
| `required`                    | `["city"]`                           | Marks the key as mandatory; the model won’t try to omit or rename it.                                                 |
| `additionalProperties`        | `False`                              | Blocks unexpected keys (e.g., `"country"`, `"unit"`), which could crash your function.                                |


## Testing it (Optional)

If you have OpenAI API Key (or alternative provider), let's test it.

A question could be `"What's the weather like in Germany?"`

Experiment with different system prompts to have better answers
from the system.

You can use [chat_assistant.py](https://github.com/alexeygrigorev/rag-agents-workshop/blob/main/chat_assistant.py)
or implement everything yourself 


In [4]:
!wget https://raw.githubusercontent.com/alexeygrigorev/rag-agents-workshop/refs/heads/main/chat_assistant.py

--2025-07-22 21:08:13--  https://raw.githubusercontent.com/alexeygrigorev/rag-agents-workshop/refs/heads/main/chat_assistant.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3485 (3,4K) [text/plain]
Saving to: ‘chat_assistant.py’

chat_assistant.py   100%[===================>]   3,40K  --.-KB/s    in 0s      

2025-07-22 21:08:14 (21,0 MB/s) - ‘chat_assistant.py’ saved [3485/3485]

